In [ ]:
import requests
import pandas as pd

from opentab_api import get_all_feedback, should_have_submitted_feedback, get_feedback_table
from reports import feedback_shame_table

In [ ]:
df_feedback = get_feedback_table()
df_feedback.groupby('target').agg({'source':'count', 'target':'first'}).sort_values('source')

In [ ]:
df_shame_all = pd.concat([feedback_shame_table(round=rnd) for rnd in ['r/1', 'r/2', 'r/3']]).groupby('name').agg({'missing': 'sum', 'name':'first'})
df_shame_all.sort_values('missing', ascending=False).to_csv('shame.csv')

In [ ]:
df_should_submit, df_submits_for, df_names = should_have_submitted_feedback(round='r/1')
df_names

In [ ]:
df_feedback.join(df_names, 'source', rsuffix='_src').join(df_names, 'target', rsuffix='_target').loc[:,['name','name_target', 'comments']].to_excel('fb.xlsx')

In [ ]:
df_fb_comp = df_feedback.groupby('target').agg({'target': 'first', 'competency': 'mean', 'should_break': 'mean'}).set_index('target')

In [ ]:
df_fb_comp.join(df_names).sort_values('competency', ascending=False)

In [ ]:
ROUND = 'r/3'

In [ ]:
draw = requests.get(f'http://localhost:5000/{ROUND}/draw').json()['payload']

result = []

for room in draw['draw']['rooms']:
    for judge_link in room['judges']:
        judge = draw['entities'][judge_link['href']]
        ca = len([inst for inst in judge['institutions'] if inst['name'] == "CA"])
        result.append({'room': room['href'], 'room_name': room['venue']['name'], 'judge': judge_link['href'], 'judge_name': judge['displayName'], 'wing_score': judge['wingCompetenceLevel'], 'chair_score': judge['chairCompetenceLevel'], 'ca': ca})

df_rooms = pd.DataFrame(result).set_index('judge')

In [ ]:
df_fb_comp

In [ ]:
df_rooms = df_rooms.join(df_fb_comp)

In [ ]:
df_rooms.groupby('room').agg({'room': 'first', 'room_name': 'first', 'wing_score': 'mean', 'chair_score': 'max', 'judge_name': 'count', 'ca': 'sum', 'competency': 'mean'})

In [ ]:
df_rooms[df_rooms.room == 'd/16']